<a href="https://colab.research.google.com/github/dsrojo10/Scraping_FPC_Wikipedia/blob/main/Scrping_Todos_contra_Todos(2004_2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Fútbol Profesional Colombiano (2004 - 2024):
Información extraida de Wikipedia API.

### Pruebas API:

In [1]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.3-py3-none-any.whl size=14325 sha256=75a0004a1513f20faa8535c9b748ee39bc18d0524b9b6077972eb19027f07cf6
  Stored in directory: /root/.cache/pip/wheels/69/6a/a2/44de9900e7f3efebd840956f4c2f82f000235ead32005fd74e
Successfully built wikipedia-api


In [5]:
import wikipediaapi

def get_wikipedia_article(article_title):
    # Crear una instancia de la API de Wikipedia en español con un agente de usuario específico
    wiki_wiki = wikipediaapi.Wikipedia(
        language='es',
        user_agent='mi_agente_de_usuario/1.0'  # Especificar el user_agent correctamente
    )

    # Obtener el artículo
    page = wiki_wiki.page(article_title)

    # Verificar si el artículo existe
    if page.exists():
        print(f"Título: {page.title}")
        print(f"Resumen: {page.summary[:500]}...")  # Mostrar los primeros 500 caracteres del resumen
    else:
        print("El artículo no existe.")

# Ejemplo de uso
get_wikipedia_article("Python (lenguaje de programación)")

Título: Python
Resumen: Python es un lenguaje de alto nivel de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código. Se trata de un lenguaje de programación multiparadigma, ya que soporta parcialmente la orientación a objetos, programación imperativa y, en menor medida, programación funcional. Es un lenguaje interpretado, dinámico y multiplataforma.
Administrado por Python Software Foundation, posee una licencia de código abierto, denominada Python Software Foundation License.[3]​ Pytho...


### Prueba FPC:

In [9]:
import requests
from bs4 import BeautifulSoup

def obtener_resultados_wikipedia(url):
    # Realizar la solicitud HTTP a la página de Wikipedia
    response = requests.get(url)

    # Verificar que la solicitud fue exitosa
    if response.status_code != 200:
        print("Error al acceder a la página de Wikipedia")
        return

    # Analizar el contenido HTML de la página
    soup = BeautifulSoup(response.content, 'html.parser')

    # Buscar la sección de "Resultados"
    resultados_section = soup.find('h3', id='Resultados')

    if not resultados_section:
        print("No se encontró la sección de 'Resultados'")
        return

    # Obtener la tabla de resultados que sigue a la sección de "Resultados"
    resultados_table = resultados_section.find_next('table')

    if not resultados_table:
        print("No se encontró la tabla de resultados")
        return

    # Extraer los datos de la tabla de resultados
    resultados = []
    for tr in resultados_table.find_all('tr'):
        tds = tr.find_all('td')
        if len(tds) == 7:
            resultado = {
                'Local': tds[0].get_text(strip=True),
                'Resultado': tds[1].get_text(strip=True),
                'Visitante': tds[2].get_text(strip=True),
                'Estadio': tds[3].get_text(strip=True),
                'Fecha': tds[4].get_text(strip=True),
                'Hora': tds[5].get_text(strip=True),
                'Transmisión': tds[6].get_text(strip=True)
            }
            resultados.append(resultado)

    # Mostrar los resultados
    for resultado in resultados:
        print(f"Local: {resultado['Local']}")
        print(f"Resultado: {resultado['Resultado']}")
        print(f"Visitante: {resultado['Visitante']}")
        print(f"Estadio: {resultado['Estadio']}")
        print(f"Fecha: {resultado['Fecha']}")
        print(f"Hora: {resultado['Hora']}")
        print(f"Transmisión: {resultado['Transmisión']}")
        print()

# URL de la página de Wikipedia
url = "https://es.wikipedia.org/wiki/Torneo_Finalizaci%C3%B3n_2024_(Colombia)"

# Obtener la información de la sección de "Resultados"
obtener_resultados_wikipedia(url)

Local: Boyacá Chicó
Resultado: 0 : 1
Visitante: Once Caldas
Estadio: La Independencia
Fecha: 15 de julio
Hora: 20:15
Transmisión: Win Sports y Win+ Fútbol

Local: Santa Fe
Resultado: 2 : 1
Visitante: Deportivo Pasto
Estadio: Nemesio Camacho El Campín
Fecha: 17 de julio
Hora: 18:00
Transmisión: Win Sports y Win+ Fútbol

Local: Jaguares
Resultado: 0 : 0
Visitante: Patriotas
Estadio: Jaraguay
Fecha: 19 de julio
Hora: 17:00
Transmisión: Win Sports y Win+ Fútbol



# Codigo Final: Todos contra Todos

## 2011 - 2024:

In [126]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

def obtener_resultados_wikipedia(url):
    response = requests.get(url)

    if response.status_code != 200:
        print("Error al acceder a la página de Wikipedia")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Obtener el nombre del torneo desde el título de la página
    titulo_torneo = soup.find('span', {'class': 'mw-page-title-main'})
    if titulo_torneo:
        torneo_info = titulo_torneo.get_text(strip=True)
        # Extraer el año y tipo de torneo del nombre
        partes = torneo_info.split()
        if len(partes) >= 3:
            tipo_torneo = partes[1]  # 'Apertura', 'Finalización' o 'Campeonato'
            año_torneo = partes[2]   # El año (como '2002')
        else:
            tipo_torneo = 'Desconocido'
            año_torneo = 'Desconocido'
    else:
        tipo_torneo = 'Desconocido'
        año_torneo = 'Desconocido'

    tablas = soup.find_all('table', {'class': 'wikitable collapsible collapsed'})

    if not tablas:
        print("No se encontraron tablas de resultados")
        return None

    print(f"Tablas encontradas: {len(tablas)}")

    data = []

    for tabla in tablas:
        # Obtener el nombre de la fecha
        fecha_header = tabla.find('th', {'colspan': '12'})
        if not fecha_header:
            continue
        fecha_link = fecha_header.find('a')
        fecha_nombre = fecha_link.get_text(strip=True) if fecha_link else fecha_header.get_text(strip=True)

        print(f"Procesando {fecha_nombre}")

        # Obtener todas las filas de partidos
        filas = tabla.find_all('tr')[2:]  # Ignorar encabezado y nombres de columnas

        for fila in filas:
            celdas = fila.find_all(['td', 'th'])

            if len(celdas) >= 4:  # Asegurar que es una fila de partido válida
                try:
                    # Obtener solo las primeras cuatro columnas
                    local = celdas[0].get_text(strip=True)
                    resultado = celdas[1].get_text(strip=True).replace('\xa0', ' ')
                    visitante = celdas[2].get_text(strip=True)
                    estadio = celdas[3].get_text(strip=True)

                    partido = {
                        'Fecha': fecha_nombre,
                        'Local': local,
                        'Resultado': resultado,
                        'Visitante': visitante,
                        'Estadio': estadio,
                        'Año': año_torneo,
                        'Tipo de Torneo': tipo_torneo
                    }

                    if fecha_nombre != 'Resultados':
                        data.append(partido)

                except Exception as e:
                    print(f"Error procesando partido en {fecha_nombre}: {e}")
                    continue

    if not data:
        print("No se encontraron datos para procesar")
        return None

    df = pd.DataFrame(data)

    # Verificaciones
    num_fechas = len(df['Fecha'].unique())
    partidos_por_fecha = df.groupby('Fecha').size()

    print(f"\nNúmero total de fechas encontradas: {num_fechas}")
    print("\nPartidos por fecha:")
    print(partidos_por_fecha)

    # Obtener el nombre de la URL para el archivo CSV
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path.split('/')[-1]  # Obtener la parte después de 'wiki/'
    filename = urllib.parse.unquote(path)  # Decodificar URL para obtener el nombre correcto

    # Crear la carpeta 'todos_contra_todos' si no existe
    carpeta = 'todos_contra_todos'
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Guardar en CSV dentro de la carpeta especificada
    file_path = os.path.join(carpeta, f'{filename}.csv')
    df.to_csv(file_path, index=False)
    print(f"\nLos resultados se han guardado en '{file_path}'")

    return df


### Prueba unitaria:

In [125]:
# URL de la página de Wikipedia
# url = "https://es.wikipedia.org/wiki/Torneo_Finalizaci%C3%B3n_2024_(Colombia)"
url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2024_(Colombia)"
# url = "https://es.wikipedia.org/wiki/Campeonato_colombiano_2020"

# url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2015_(Colombia)"

print("Iniciando script...")
# Obtener la información
df_resultados = obtener_resultados_wikipedia(url)

if df_resultados is not None:
    print("\nPrimeros registros del DataFrame:")
    print(df_resultados.head())
else:
    print("No se pudo obtener el DataFrame")


Iniciando script...
Tablas encontradas: 21
Procesando Fecha 1
Procesando Fecha 2
Procesando Fecha 3
Procesando Fecha 4
Procesando Fecha 5
Procesando Fecha 6
Procesando Fecha 7
Procesando Fecha 8
Procesando Fecha 9
Procesando Fecha 10
Procesando Fecha 11
Procesando Fecha 12
Procesando Fecha 13
Procesando Fecha 14
Procesando Fecha 15
Procesando Fecha 16
Procesando Fecha 17
Procesando Fecha 18
Procesando Fecha 19
Procesando Resultados
Procesando Resultados

Número total de fechas encontradas: 19

Partidos por fecha:
Fecha
Fecha 1     10
Fecha 10    10
Fecha 11    10
Fecha 12    10
Fecha 13    10
Fecha 14    10
Fecha 15    10
Fecha 16    10
Fecha 17    10
Fecha 18    10
Fecha 19    10
Fecha 2     10
Fecha 3     10
Fecha 4     10
Fecha 5     10
Fecha 6     10
Fecha 7     10
Fecha 8     10
Fecha 9     10
dtype: int64

Los resultados se han guardado en 'todos_contra_todos1/Torneo_Apertura_2024_(Colombia).csv'

Primeros registros del DataFrame:
     Fecha              Local Resultado        Vi

### Automatización:

In [32]:
def procesar_urls_desde_archivo():
    # Leer las URLs desde el archivo urls.txt
    with open('url1.txt', 'r') as f:
        urls = f.readlines()

    for url in urls:
        url = url.strip()  # Eliminar posibles saltos de línea y espacios en blanco
        print(f"Procesando URL: {url}")

        df_resultados = obtener_resultados_wikipedia(url)

        if df_resultados is not None:
            print("\nPrimeros registros del DataFrame:")
            print(df_resultados.head())
        else:
            print(f"No se pudo obtener el DataFrame para {url}")

# Ejecutar el procesamiento de todas las URLs
procesar_urls_desde_archivo()

Procesando URL: https://es.wikipedia.org/wiki/Torneo_Finalización_2011_(Colombia)
Tablas encontradas: 18
Procesando Fecha 1
Procesando Fecha 2
Procesando Fecha 3
Procesando Fecha 4
Procesando Fecha 5
Procesando Fecha 6
Procesando Fecha 7
Procesando Fecha 8
Procesando Fecha 9
Procesando Fecha 10
Procesando Fecha 11
Procesando Fecha 12
Procesando Fecha 13
Procesando Fecha 14
Procesando Fecha 15
Procesando Fecha 16
Procesando Fecha 17
Procesando Fecha 18

Número total de fechas encontradas: 18

Partidos por fecha:
Fecha
Fecha 1     9
Fecha 10    9
Fecha 11    9
Fecha 12    9
Fecha 13    9
Fecha 14    9
Fecha 15    9
Fecha 16    9
Fecha 17    9
Fecha 18    9
Fecha 2     9
Fecha 3     9
Fecha 4     9
Fecha 5     9
Fecha 6     9
Fecha 7     9
Fecha 8     9
Fecha 9     9
dtype: int64

Los resultados se han guardado en 'todos_contra_todos/Torneo_Finalización_2011_(Colombia).csv'

Primeros registros del DataFrame:
     Fecha                   Local Resultado          Visitante  \
0  Fecha 1  In

In [16]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

def obtener_resultados_wikipedia(url):
    response = requests.get(url)

    if response.status_code != 200:
        print("Error al acceder a la página de Wikipedia")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Obtener el nombre del torneo desde el título de la página
    titulo_torneo = soup.find('span', {'class': 'mw-page-title-main'})
    if titulo_torneo:
        torneo_info = titulo_torneo.get_text(strip=True)
        # Extraer el año y tipo de torneo del nombre
        partes = torneo_info.split()
        if len(partes) >= 3:
            tipo_torneo = partes[1]  # 'Apertura', 'Finalización' o 'Campeonato'
            año_torneo = partes[2]   # El año (como '2002')
        else:
            tipo_torneo = 'Desconocido'
            año_torneo = 'Desconocido'
    else:
        tipo_torneo = 'Desconocido'
        año_torneo = 'Desconocido'

    # Buscar tanto tablas colapsadas como no colapsadas
    tablas = soup.find_all('table', {'class': ['wikitable collapsible collapsed',
                                              'wikitable mw-collapsible mw-collapsed',
                                              'wikitable mw-made-collapsible']})

    if not tablas:
        print("No se encontraron tablas de resultados")
        return None

    print(f"Tablas encontradas: {len(tablas)}")

    data = []

    for tabla in tablas:
        # Obtener el nombre de la fecha
        fecha_header = tabla.find('th', {'colspan': True})  # Buscar th con colspan
        if not fecha_header:
            continue

        # Buscar el enlace dentro del encabezado
        fecha_link = fecha_header.find('a')
        fecha_nombre = fecha_link.get_text(strip=True) if fecha_link else fecha_header.get_text(strip=True)

        # Limpiar el texto "mostrar" u "ocultar" si existe
        fecha_nombre = fecha_nombre.replace('mostrar', '').replace('ocultar', '').strip()

        # Validar si la fecha comienza con "Fecha"
        if not fecha_nombre.startswith('Fecha'):
            continue

        print(f"Procesando {fecha_nombre}")

        # Obtener todas las filas de partidos (ignorando encabezados)
        filas = tabla.find_all('tr')[2:]  # Ignorar encabezado y nombres de columnas

        for fila in filas:
            celdas = fila.find_all(['td', 'th'])

            if len(celdas) >= 4:  # Asegurar que es una fila de partido válida
                try:
                    # Obtener datos básicos del partido
                    local = celdas[0].get_text(strip=True)
                    resultado = celdas[1].get_text(strip=True).replace('\xa0', ' ')
                    visitante = celdas[2].get_text(strip=True)
                    estadio = celdas[3].get_text(strip=True)

                    # Limpiar los datos si es necesario
                    resultado = resultado.replace(':', ' : ')  # Estandarizar formato de resultado

                    partido = {
                        'Fecha': fecha_nombre,
                        'Local': local,
                        'Resultado': resultado,
                        'Visitante': visitante,
                        'Estadio': estadio,
                        'Año': año_torneo,
                        'Tipo de Torneo': tipo_torneo
                    }

                    # Solo agregar si todos los campos necesarios están presentes
                    if local and visitante and fecha_nombre:
                        data.append(partido)

                except Exception as e:
                    print(f"Error procesando partido en {fecha_nombre}: {e}")
                    continue

    if not data:
        print("No se encontraron datos para procesar")
        return None

    df = pd.DataFrame(data)

    # Verificaciones
    num_fechas = len(df['Fecha'].unique())
    partidos_por_fecha = df.groupby('Fecha').size()

    print(f"\nNúmero total de fechas encontradas: {num_fechas}")
    print("\nPartidos por fecha:")
    print(partidos_por_fecha)

    # Obtener el nombre de la URL para el archivo CSV
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path.split('/')[-1]  # Obtener la parte después de 'wiki/'
    filename = urllib.parse.unquote(path)  # Decodificar URL para obtener el nombre correcto

    # Crear la carpeta si no existe
    carpeta = 'prueba'
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Guardar en CSV dentro de la carpeta especificada
    file_path = os.path.join(carpeta, f'{filename}.csv')
    df.to_csv(file_path, index=False)
    print(f"\nLos resultados se han guardado en '{file_path}'")

    return df

In [20]:
# URL de la página de Wikipedia
# url = "https://es.wikipedia.org/wiki/Torneo_Finalizaci%C3%B3n_2024_(Colombia)"
# url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2024_(Colombia)"
# url = "https://es.wikipedia.org/wiki/Campeonato_colombiano_2020"

# url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2015_(Colombia)"

print("Iniciando script...")
# Obtener la información
df_resultados = obtener_resultados_wikipedia(url)

if df_resultados is not None:
    print("\nPrimeros registros del DataFrame:")
    print(df_resultados.head())
else:
    print("No se pudo obtener el DataFrame")


Iniciando script...
Tablas encontradas: 23
Procesando Fecha 1
Procesando Fecha 2
Procesando Fecha 3
Procesando Fecha 4
Procesando Fecha 5
Procesando Fecha 6
Procesando Fecha 7
Procesando Fecha 8
Procesando Fecha 9
Procesando Fecha 10
Procesando Fecha 11
Procesando Fecha 12
Procesando Fecha 13
Procesando Fecha 14
Procesando Fecha 15
Procesando Fecha 16
Procesando Fecha 17
Procesando Fecha 18
Procesando Fecha 19
Procesando Fecha 20

Número total de fechas encontradas: 20

Partidos por fecha:
Fecha
Fecha 1     10
Fecha 10    10
Fecha 11    10
Fecha 12    10
Fecha 13    10
Fecha 14    10
Fecha 15    10
Fecha 16    10
Fecha 17    10
Fecha 18    10
Fecha 19    10
Fecha 2     10
Fecha 20    10
Fecha 3     10
Fecha 4     10
Fecha 5     10
Fecha 6     10
Fecha 7     10
Fecha 8     10
Fecha 9     10
dtype: int64

Los resultados se han guardado en 'prueba/Campeonato_colombiano_2020.csv'

Primeros registros del DataFrame:
     Fecha                 Local      Resultado               Visitante  \
0

## 2004, 2005, 2008:

In [64]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

def obtener_resultados_wikipedia_legacy(url):
    response = requests.get(url)

    if response.status_code != 200:
        print("Error al acceder a la página de Wikipedia")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Obtener el nombre del torneo desde el título de la página
    titulo_torneo = soup.find('span', {'class': 'mw-page-title-main'})
    if titulo_torneo:
        torneo_info = titulo_torneo.get_text(strip=True)
        partes = torneo_info.split()
        if len(partes) >= 3:
            tipo_torneo = partes[1]
            año_torneo = partes[2]
        else:
            tipo_torneo = 'Desconocido'
            año_torneo = 'Desconocido'
    else:
        tipo_torneo = 'Desconocido'
        año_torneo = 'Desconocido'

    # Buscar tablas con el formato antiguo
    tablas = soup.find_all('table', {'bgcolor': '#f7f8ff'})

    if not tablas:
        print("No se encontraron tablas de resultados")
        return None

    print(f"Tablas encontradas: {len(tablas)}")

    data = []

    for tabla in tablas:
        # Obtener el encabezado de la fecha (ahora en formato "Fecha X - DD y DD de MM de AAAA")
        fecha_header = tabla.find('th', {'colspan': '4'})
        if not fecha_header:
            continue

        fecha_nombre = fecha_header.get_text(strip=True)
        if not fecha_nombre.startswith('Fecha'):
            continue

        # Extraer solo el número de la fecha
        fecha_numero = fecha_nombre.split('-')[0].strip()
        print(f"Procesando {fecha_numero}")

        # Obtener las filas de partidos
        filas = tabla.find_all('tr')[2:]  # Ignorar encabezados

        fecha_actual = None
        for fila in filas:
            celdas = fila.find_all(['td'])

            if len(celdas) >= 3:  # Verificar que tengamos suficientes celdas
                try:
                    # Actualizar la fecha_actual si existe una celda con rowspan
                    fecha_cell = fila.find('td', {'rowspan': True})
                    if fecha_cell:
                        fecha_actual = fecha_cell.get_text(strip=True)

                    # Determinar el índice correcto para cada columna
                    if fecha_cell:
                        # Si hay celda de fecha, los índices empiezan después
                        local = celdas[1].get_text(strip=True)
                        resultado = celdas[2].get_text(strip=True)
                        visitante = celdas[3].get_text(strip=True)
                    else:
                        # Si no hay celda de fecha, usar los índices normales
                        local = celdas[0].get_text(strip=True)
                        resultado = celdas[1].get_text(strip=True)
                        visitante = celdas[2].get_text(strip=True)

                    # Limpiar el formato del resultado
                    resultado = resultado.replace('-', ' - ')

                    # Limpiar nombres de equipos (quitar negrita)
                    local = local.replace('b>', '').replace('</', '').replace('<', '')
                    visitante = visitante.replace('b>', '').replace('</', '').replace('<', '')

                    partido = {
                        'Fecha': fecha_numero,
                        'Día': fecha_actual,
                        'Local': local,
                        'Resultado': resultado,
                        'Visitante': visitante,
                        'Año': año_torneo,
                        'Tipo de Torneo': tipo_torneo
                    }

                    if local and visitante and fecha_actual:
                        data.append(partido)

                except Exception as e:
                    print(f"Error procesando partido en {fecha_numero}: {e}")
                    continue

    if not data:
        print("No se encontraron datos para procesar")
        return None

    df = pd.DataFrame(data)

    # Verificaciones
    num_fechas = len(df['Fecha'].unique())
    partidos_por_fecha = df.groupby('Fecha').size()

    print(f"\nNúmero total de fechas encontradas: {num_fechas}")
    print("\nPartidos por fecha:")
    print(partidos_por_fecha)

    # Guardar en CSV
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path.split('/')[-1]
    filename = urllib.parse.unquote(path)

  # Crear la carpeta 'todos_contra_todos' si no existe
    carpeta = 'todos_contra_todos'
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    file_path = os.path.join(carpeta, f'{filename}_legacy.csv')
    df.to_csv(file_path, index=False)
    print(f"\nLos resultados se han guardado en '{file_path}'")

    return df

In [57]:
def procesar_urls_desde_archivo():
    # Leer las URLs desde el archivo urls.txt
    with open('url2.txt', 'r') as f:
        urls = f.readlines()

    for url in urls:
        url = url.strip()  # Eliminar posibles saltos de línea y espacios en blanco
        print(f"Procesando URL: {url}")

        df_resultados = obtener_resultados_wikipedia_legacy(url)

        if df_resultados is not None:
            print("\nPrimeros registros del DataFrame:")
            print(df_resultados.head())
        else:
            print(f"No se pudo obtener el DataFrame para {url}")

# Ejecutar el procesamiento de todas las URLs
procesar_urls_desde_archivo()

Procesando URL: https://es.wikipedia.org/wiki/Torneo_Finalización_2004_(Colombia)
Tablas encontradas: 31
Procesando Fecha 1
Procesando Fecha 2
Procesando Fecha 3
Procesando Fecha 4
Procesando Fecha 5
Procesando Fecha 6
Procesando Fecha 7
Procesando Fecha 8
Procesando Fecha 9
Procesando Fecha 10
Procesando Fecha 11
Procesando Fecha 12
Procesando Fecha 13
Procesando Fecha 14
Procesando Fecha 15
Procesando Fecha 16
Procesando Fecha 17
Procesando Fecha 18

Número total de fechas encontradas: 18

Partidos por fecha:
Fecha
Fecha 1     9
Fecha 10    9
Fecha 11    9
Fecha 12    9
Fecha 13    9
Fecha 14    9
Fecha 15    9
Fecha 16    9
Fecha 17    9
Fecha 18    9
Fecha 2     9
Fecha 3     9
Fecha 4     9
Fecha 5     9
Fecha 6     9
Fecha 7     9
Fecha 8     9
Fecha 9     9
dtype: int64

Los resultados se han guardado en 'todos_contra_todos/Torneo_Finalización_2004_(Colombia)_legacy.csv'

Primeros registros del DataFrame:
     Fecha          Día             Local Resultado     Visitante   Año  \

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2006-2007

In [76]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

def obtener_resultados_wikipedia_legacy(url):
    response = requests.get(url)

    if response.status_code != 200:
        print("Error al acceder a la página de Wikipedia")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Obtener el nombre del torneo desde el título de la página
    titulo_torneo = soup.find('span', {'class': 'mw-page-title-main'})
    if titulo_torneo:
        torneo_info = titulo_torneo.get_text(strip=True)
        partes = torneo_info.split()
        if len(partes) >= 3:
            tipo_torneo = partes[1]
            año_torneo = partes[2]
        else:
            tipo_torneo = 'Desconocido'
            año_torneo = 'Desconocido'
    else:
        tipo_torneo = 'Desconocido'
        año_torneo = 'Desconocido'

    print("Iniciando búsqueda de tablas...")

    # Buscar las columnas que contienen las tablas de fechas
    columnas = soup.find_all('td', {'valign': 'top', 'width': '50%'})

    if not columnas:
        print("No se encontraron columnas con tablas")
        return None

    print(f"Columnas encontradas: {len(columnas)}")

    data = []
    fechas_procesadas = set()

    for columna in columnas:
        tabla = columna.find('table', {'bgcolor': '#f7f8ff'})
        if not tabla:
            continue

        # Obtener el encabezado de la fecha
        fecha_header = tabla.find('th', {'colspan': '3'})
        if not fecha_header:
            continue

        fecha_nombre = fecha_header.get_text(strip=True)
        if not fecha_nombre.startswith('Fecha'):
            continue

        # Extraer solo el número de la fecha
        fecha_numero = fecha_nombre.split('-')[0].strip()

        # Si ya procesamos esta fecha, la saltamos
        if fecha_numero in fechas_procesadas:
            continue

        print(f"Procesando {fecha_numero}")

        # Obtener las filas de partidos (ignorar las dos primeras filas: header de fecha y header de columnas)
        filas = tabla.find_all('tr')[2:]  # Ignorar los dos encabezados

        partidos_en_fecha = []  # Lista para almacenar los partidos de esta fecha

        for fila in filas:
            celdas = fila.find_all(['td'])

            if len(celdas) == 3:  # Exactamente 3 celdas
                try:
                    local = celdas[0].get_text(strip=True)
                    resultado = celdas[1].get_text(strip=True)
                    visitante = celdas[2].get_text(strip=True)

                    # Verificar que tengamos datos válidos
                    if local and visitante and resultado:
                        fecha_completa = fecha_header.get_text(strip=True).split('-')[1].strip()

                        # Limpiar el formato del resultado
                        resultado = resultado.replace('-', ' - ')

                        # Limpiar nombres de equipos (quitar negrita)
                        local = local.replace('b>', '').replace('</', '').replace('<', '')
                        visitante = visitante.replace('b>', '').replace('</', '').replace('<', '')

                        partido = {
                            'Fecha': fecha_numero,
                            'Día': fecha_completa,
                            'Local': local,
                            'Resultado': resultado,
                            'Visitante': visitante,
                            'Año': año_torneo,
                            'Tipo de Torneo': tipo_torneo
                        }

                        partidos_en_fecha.append(partido)

                except Exception as e:
                    print(f"Error procesando partido en {fecha_numero}: {e}")
                    continue

        # Verificar que tengamos exactamente 9 partidos antes de agregarlos
        if len(partidos_en_fecha) == 9:
            data.extend(partidos_en_fecha)
            fechas_procesadas.add(fecha_numero)
            print(f"Fecha {fecha_numero} procesada correctamente con 9 partidos")
        else:
            print(f"Advertencia: La {fecha_numero} tiene {len(partidos_en_fecha)} partidos en lugar de 9")

    if not data:
        print("No se encontraron datos para procesar")
        return None

    df = pd.DataFrame(data)

    # Verificaciones
    num_fechas = len(df['Fecha'].unique())
    partidos_por_fecha = df.groupby('Fecha').size()

    print(f"\nNúmero total de fechas encontradas: {num_fechas}")
    print("\nPartidos por fecha:")
    print(partidos_por_fecha)

    # Guardar en CSV
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path.split('/')[-1]
    filename = urllib.parse.unquote(path)

    carpeta = 'todos_contra_todos_2006-07'
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    file_path = os.path.join(carpeta, f'{filename}_legacy.csv')
    df.to_csv(file_path, index=False)
    print(f"\nLos resultados se han guardado en '{file_path}'")

    return df

In [77]:
def procesar_urls_desde_archivo():
    # Leer las URLs desde el archivo urls.txt
    with open('url3.txt', 'r') as f:
        urls = f.readlines()

    for url in urls:
        url = url.strip()  # Eliminar posibles saltos de línea y espacios en blanco
        print(f"Procesando URL: {url}")

        df_resultados = obtener_resultados_wikipedia_legacy(url)

        if df_resultados is not None:
            print("\nPrimeros registros del DataFrame:")
            print(df_resultados.head())
        else:
            print(f"No se pudo obtener el DataFrame para {url}")

# Ejecutar el procesamiento de todas las URLs
procesar_urls_desde_archivo()

Procesando URL: https://es.wikipedia.org/wiki/Torneo_Finalización_2006_(Colombia)
Iniciando búsqueda de tablas...
Columnas encontradas: 30
Procesando Fecha 1
Fecha Fecha 1 procesada correctamente con 9 partidos
Procesando Fecha 2
Fecha Fecha 2 procesada correctamente con 9 partidos
Procesando Fecha 3
Fecha Fecha 3 procesada correctamente con 9 partidos
Procesando Fecha 4
Fecha Fecha 4 procesada correctamente con 9 partidos
Procesando Fecha 5
Fecha Fecha 5 procesada correctamente con 9 partidos
Procesando Fecha 6
Fecha Fecha 6 procesada correctamente con 9 partidos
Procesando Fecha 7
Fecha Fecha 7 procesada correctamente con 9 partidos
Procesando Fecha 8
Fecha Fecha 8 procesada correctamente con 9 partidos
Procesando Fecha 9
Fecha Fecha 9 procesada correctamente con 9 partidos
Procesando Fecha 10
Fecha Fecha 10 procesada correctamente con 9 partidos
Procesando Fecha 11
Fecha Fecha 11 procesada correctamente con 9 partidos
Procesando Fecha 12
Fecha Fecha 12 procesada correctamente con 9 

In [74]:
# URL de la página de Wikipedia
# url = "https://es.wikipedia.org/wiki/Torneo_Finalizaci%C3%B3n_2024_(Colombia)"
# url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2024_(Colombia)"
# url = "https://es.wikipedia.org/wiki/Campeonato_colombiano_2020"

# url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2004_(Colombia)"
# url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2006_(Colombia)"
url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2007_(Colombia)"

print("Iniciando script...")
# Obtener la información
df_resultados = obtener_resultados_wikipedia_legacy(url)

if df_resultados is not None:
    print("\nPrimeros registros del DataFrame:")
    print(df_resultados.head())
else:
    print("No se pudo obtener el DataFrame")


Iniciando script...
Iniciando búsqueda de tablas...
Columnas encontradas: 30
Procesando Fecha 1
Fecha Fecha 1 procesada correctamente con 9 partidos
Procesando Fecha 2
Fecha Fecha 2 procesada correctamente con 9 partidos
Procesando Fecha 3
Fecha Fecha 3 procesada correctamente con 9 partidos
Procesando Fecha 4
Fecha Fecha 4 procesada correctamente con 9 partidos
Procesando Fecha 5
Fecha Fecha 5 procesada correctamente con 9 partidos
Procesando Fecha 6
Fecha Fecha 6 procesada correctamente con 9 partidos
Procesando Fecha 7
Fecha Fecha 7 procesada correctamente con 9 partidos
Procesando Fecha 8
Fecha Fecha 8 procesada correctamente con 9 partidos
Procesando Fecha 9
Fecha Fecha 9 procesada correctamente con 9 partidos
Procesando Fecha 10
Fecha Fecha 10 procesada correctamente con 9 partidos
Procesando Fecha 11
Fecha Fecha 11 procesada correctamente con 9 partidos
Procesando Fecha 12
Fecha Fecha 12 procesada correctamente con 9 partidos
Procesando Fecha 13
Fecha Fecha 13 procesada correcta

In [78]:
# prompt: descargar todo lo de la carpeta todos_contra_todos_2006-07

import shutil
import os

def zip_directory(directory_path, zip_filename):
    """Zips a directory and its contents."""
    try:
        shutil.make_archive(zip_filename, 'zip', directory_path)
        print(f"Directory '{directory_path}' successfully zipped to '{zip_filename}.zip'")
    except FileNotFoundError:
        print(f"Error: Directory '{directory_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the directory to zip and the desired zip filename
directory_to_zip = "todos_contra_todos_2006-07"
zip_file_name = "todos_contra_todos_2006-07"

# Create the zip file
zip_directory(directory_to_zip, zip_file_name)

# Download the zip file (if the directory exists)
if os.path.exists(f"{zip_file_name}.zip"):
    from google.colab import files
    files.download(f"{zip_file_name}.zip")
else:
    print("Zip file not found. Check if the directory exists and was zipped correctly.")

Directory 'todos_contra_todos_2006-07' successfully zipped to 'todos_contra_todos_2006-07.zip'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 2009:


In [91]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

def obtener_resultados_wikipedia_2009(url):
    response = requests.get(url)

    if response.status_code != 200:
        print("Error al acceder a la página de Wikipedia")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Obtener el nombre del torneo desde el título de la página
    titulo_torneo = soup.find('span', {'class': 'mw-page-title-main'})
    if titulo_torneo:
        torneo_info = titulo_torneo.get_text(strip=True)
        partes = torneo_info.split()
        if len(partes) >= 3:
            tipo_torneo = partes[1]
            año_torneo = partes[2]
        else:
            tipo_torneo = 'Desconocido'
            año_torneo = 'Desconocido'
    else:
        tipo_torneo = 'Desconocido'
        año_torneo = 'Desconocido'

    print("Iniciando búsqueda de tablas...")

    # Buscar las columnas que contienen las tablas de fechas
    columnas = soup.find_all('td', {'valign': 'top', 'width': '50%'})

    if not columnas:
        print("No se encontraron columnas con tablas")
        return None

    print(f"Columnas encontradas: {len(columnas)}")

    data = []
    fechas_procesadas = set()

    for columna in columnas:
        tabla = columna.find('table', {'bgcolor': '#f7f8ff'})
        if not tabla:
            continue

        # Obtener el encabezado de la fecha
        fecha_header = tabla.find('th', {'colspan': '5'})
        if not fecha_header:
            continue

        fecha_nombre = fecha_header.get_text(strip=True)
        if not fecha_nombre.startswith('Fecha'):
            continue

        fecha_numero = fecha_nombre.split('-')[0].strip()

        if fecha_numero in fechas_procesadas:
            continue

        print(f"Procesando {fecha_numero}")

        # Obtener las filas de partidos
        filas = tabla.find_all('tr')[2:]  # Ignorar los dos encabezados
        partidos_en_fecha = []

        for fila in filas:
            celdas = fila.find_all('td')

            # Buscar las celdas que contienen la información del partido
            if len(celdas) >= 3:
                try:
                    # Si la fila tiene día y hora, los equipos están en índices diferentes
                    if 'rowspan' in str(celdas[0]):
                        if len(celdas) >= 5:
                            local = celdas[2].get_text(strip=True)
                            resultado = celdas[3].get_text(strip=True)
                            visitante = celdas[4].get_text(strip=True)
                    else:
                        # Si no hay rowspan, los equipos están en diferentes posiciones
                        if len(celdas) >= 4:
                            local = celdas[1].get_text(strip=True)
                            resultado = celdas[2].get_text(strip=True)
                            visitante = celdas[3].get_text(strip=True)
                        else:
                            local = celdas[0].get_text(strip=True)
                            resultado = celdas[1].get_text(strip=True)
                            visitante = celdas[2].get_text(strip=True)

                    if local and visitante and resultado:
                        partido = {
                            'Fecha': fecha_numero,
                            'Local': local.replace('b>', '').replace('</', '').replace('<', ''),
                            'Resultado': resultado.replace('-', ' - '),
                            'Visitante': visitante.replace('b>', '').replace('</', '').replace('<', ''),
                            'Estadio': '',  # Campo vacío para estadio
                            'Año': año_torneo,
                            'Tipo de Torneo': tipo_torneo
                        }
                        partidos_en_fecha.append(partido)

                except Exception as e:
                    print(f"Error procesando partido en {fecha_numero}: {e}")
                    continue

        if len(partidos_en_fecha) == 9:
            data.extend(partidos_en_fecha)
            fechas_procesadas.add(fecha_numero)
            print(f"Fecha {fecha_numero} procesada correctamente con 9 partidos")
        else:
            print(f"Advertencia: La {fecha_numero} tiene {len(partidos_en_fecha)} partidos en lugar de 9")

    if not data:
        print("No se encontraron datos para procesar")
        return None

    df = pd.DataFrame(data)

    # Verificaciones
    num_fechas = len(df['Fecha'].unique())
    partidos_por_fecha = df.groupby('Fecha').size()

    print(f"\nNúmero total de fechas encontradas: {num_fechas}")
    print("\nPartidos por fecha:")
    print(partidos_por_fecha)

    # Guardar en CSV
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path.split('/')[-1]
    filename = urllib.parse.unquote(path)

    carpeta = 'todos_contra_todos2009-10'
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    file_path = os.path.join(carpeta, f'{filename}_2009.csv')
    df.to_csv(file_path, index=False)
    print(f"\nLos resultados se han guardado en '{file_path}'")

    return df

In [95]:
# URL de la página de Wikipedia
# url = "https://es.wikipedia.org/wiki/Torneo_Finalizaci%C3%B3n_2024_(Colombia)"
# url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2024_(Colombia)"
# url = "https://es.wikipedia.org/wiki/Campeonato_colombiano_2020"

url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2009_(Colombia)"
# url = "https://es.wikipedia.org/wiki/Torneo_Finalizaci%C3%B3n_2009_(Colombia)"

print("Iniciando script...")
# Obtener la información
df_resultados = obtener_resultados_wikipedia_2009(url)

if df_resultados is not None:
    print("\nPrimeros registros del DataFrame:")
    print(df_resultados.head())
else:
    print("No se pudo obtener el DataFrame")


Iniciando script...
Iniciando búsqueda de tablas...
No se encontraron columnas con tablas
No se pudo obtener el DataFrame


## 2010:

In [103]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

def obtener_resultados_wikipedia_2010(url):
    response = requests.get(url)

    if response.status_code != 200:
        print("Error al acceder a la página de Wikipedia")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Obtener el nombre del torneo desde el título de la página
    titulo_torneo = soup.find('span', {'class': 'mw-page-title-main'})
    if titulo_torneo:
        torneo_info = titulo_torneo.get_text(strip=True)
        partes = torneo_info.split()
        if len(partes) >= 3:
            tipo_torneo = partes[1]
            año_torneo = partes[2]
        else:
            tipo_torneo = 'Desconocido'
            año_torneo = 'Desconocido'
    else:
        tipo_torneo = 'Desconocido'
        año_torneo = 'Desconocido'

    print("Iniciando búsqueda de tablas...")

    # Buscar tablas de resultados
    tablas = soup.select('table.wikitable')

    if not tablas:
        print("No se encontraron tablas de resultados")
        return None

    print(f"Tablas encontradas: {len(tablas)}")

    data = []
    fechas_procesadas = set()

    for tabla in tablas:
        try:
            # Obtener el encabezado de la fecha
            fecha_header = tabla.select_one('tr th[colspan="12"]')
            if not fecha_header:
                continue

            fecha_texto = fecha_header.get_text(strip=True)
            if not fecha_texto.startswith('Fecha'):
                continue

            fecha_numero = fecha_texto.split('-')[0].strip()

            if fecha_numero in fechas_procesadas:
                continue

            print(f"Procesando {fecha_numero}")

            # Obtener las filas de partidos (ignorando encabezados)
            filas_partidos = tabla.select('tr:not(:first-child):not(:nth-child(2))')
            partidos_en_fecha = []

            for fila in filas_partidos:
                celdas = fila.select('td')
                if len(celdas) >= 4:  # Necesitamos al menos las celdas esenciales
                    try:
                        # Los índices son fijos según la estructura mostrada
                        estadio_idx = 3
                        local_idx = 4
                        resultado_idx = 5
                        visitante_idx = 6

                        # Verificar si tenemos suficientes celdas
                        if len(celdas) > visitante_idx:
                            estadio = celdas[estadio_idx].get_text(strip=True)
                            local = celdas[local_idx].get_text(strip=True)
                            resultado = celdas[resultado_idx].get_text(strip=True)
                            visitante = celdas[visitante_idx].get_text(strip=True)
                        else:
                            # Para filas con rowspan, ajustar índices
                            estadio = celdas[-4].get_text(strip=True)
                            local = celdas[-3].get_text(strip=True)
                            resultado = celdas[-2].get_text(strip=True)
                            visitante = celdas[-1].get_text(strip=True)

                        # Limpiar formato
                        local = local.replace('b>', '').replace('</', '').replace('<', '')
                        visitante = visitante.replace('b>', '').replace('</', '').replace('<', '')

                        partido = {
                            'Fecha': fecha_numero,
                            'Local': local,
                            'Resultado': resultado.replace('-', ' - '),
                            'Visitante': visitante,
                            'Estadio': estadio,
                            'Año': año_torneo,
                            'Tipo de Torneo': tipo_torneo
                        }
                        partidos_en_fecha.append(partido)

                    except Exception as e:
                        print(f"Error procesando partido: {e}")
                        continue

            if len(partidos_en_fecha) == 9:
                data.extend(partidos_en_fecha)
                fechas_procesadas.add(fecha_numero)
                print(f"Fecha {fecha_numero} procesada correctamente con 9 partidos")
            else:
                print(f"Advertencia: La {fecha_numero} tiene {len(partidos_en_fecha)} partidos en lugar de 9")

        except Exception as e:
            print(f"Error procesando tabla: {e}")
            continue

    if not data:
        print("No se encontraron datos para procesar")
        return None

    df = pd.DataFrame(data)

    # Verificaciones
    num_fechas = len(df['Fecha'].unique())
    partidos_por_fecha = df.groupby('Fecha').size()

    print(f"\nNúmero total de fechas encontradas: {num_fechas}")
    print("\nPartidos por fecha:")
    print(partidos_por_fecha)

    # Guardar en CSV
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path.split('/')[-1]
    filename = urllib.parse.unquote(path)

    carpeta = 'todos_contra_todos2010'
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    file_path = os.path.join(carpeta, f'{filename}_2010.csv')
    df.to_csv(file_path, index=False)
    print(f"\nLos resultados se han guardado en '{file_path}'")

    return df

In [106]:
# url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2010_(Colombia)"
url = "https://es.wikipedia.org/wiki/Torneo_Finalizaci%C3%B3n_2010_(Colombia)"

print("Iniciando script...")
# Obtener la información
df_resultados = obtener_resultados_wikipedia_2010(url)

if df_resultados is not None:
    print("\nPrimeros registros del DataFrame:")
    print(df_resultados.head())
else:
    print("No se pudo obtener el DataFrame")


Iniciando script...
Iniciando búsqueda de tablas...
Tablas encontradas: 37
Procesando Fecha 1
Fecha Fecha 1 procesada correctamente con 9 partidos
Procesando Fecha 2
Fecha Fecha 2 procesada correctamente con 9 partidos
Procesando Fecha 3
Fecha Fecha 3 procesada correctamente con 9 partidos
Procesando Fecha 4
Fecha Fecha 4 procesada correctamente con 9 partidos
Procesando Fecha 5
Fecha Fecha 5 procesada correctamente con 9 partidos
Procesando Fecha 6
Fecha Fecha 6 procesada correctamente con 9 partidos
Procesando Fecha 7
Fecha Fecha 7 procesada correctamente con 9 partidos
Procesando Fecha 8
Fecha Fecha 8 procesada correctamente con 9 partidos
Procesando Fecha 9
Fecha Fecha 9 procesada correctamente con 9 partidos
Procesando Fecha 10
Fecha Fecha 10 procesada correctamente con 9 partidos
Procesando Fecha 11
Fecha Fecha 11 procesada correctamente con 9 partidos
Procesando Fecha 12
Fecha Fecha 12 procesada correctamente con 9 partidos
Procesando Fecha 13
Fecha Fecha 13 procesada correctame

## 2015-2019

In [133]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

def obtener_resultados_wikipedia2015(url):
    response = requests.get(url)

    if response.status_code != 200:
        print("Error al acceder a la página de Wikipedia")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Obtener el nombre del torneo desde el título de la página
    titulo_torneo = soup.find('span', {'class': 'mw-page-title-main'})
    if titulo_torneo:
        torneo_info = titulo_torneo.get_text(strip=True)
        # Extraer el año y tipo de torneo del nombre
        partes = torneo_info.split()
        if len(partes) >= 3:
            tipo_torneo = partes[1]  # 'Apertura', 'Finalización' o 'Campeonato'
            año_torneo = partes[2]   # El año (como '2002')
        else:
            tipo_torneo = 'Desconocido'
            año_torneo = 'Desconocido'
    else:
        tipo_torneo = 'Desconocido'
        año_torneo = 'Desconocido'

    # Modificación: Buscar tablas con las clases específicas de 2015
    tablas = soup.find_all('table', {'class': ['wikitable', 'mw-collapsible', 'mw-made-collapsible']})

    if not tablas:
        print("No se encontraron tablas de resultados")
        return None

    print(f"Tablas encontradas: {len(tablas)}")

    data = []

    for tabla in tablas:
        # Obtener el nombre de la fecha
        fecha_header = tabla.find('th', {'colspan': '12'})
        if not fecha_header:
            continue

        # Modificación: Buscar el enlace dentro del encabezado de manera diferente
        fecha_link = fecha_header.find('a')
        if fecha_link:
            fecha_nombre = fecha_link.get_text(strip=True)
        else:
            continue

        print(f"Procesando {fecha_nombre}")

        # Obtener todas las filas de partidos, saltando las dos primeras (encabezado y columnas)
        filas = [fila for fila in tabla.find_all('tr') if not fila.get('style') == 'display: none;']
        filas = filas[2:]  # Ignorar encabezado y nombres de columnas

        for fila in filas:
            celdas = fila.find_all(['td', 'th'])

            if len(celdas) >= 4:  # Asegurar que es una fila de partido válida
                try:
                    # Obtener solo las columnas necesarias
                    local = celdas[0].get_text(strip=True)
                    resultado = celdas[1].get_text(strip=True).replace('\xa0', ' ')
                    visitante = celdas[2].get_text(strip=True)
                    estadio = celdas[3].get_text(strip=True)

                    partido = {
                        'Fecha': fecha_nombre,
                        'Local': local,
                        'Resultado': resultado,
                        'Visitante': visitante,
                        'Estadio': estadio,
                        'Año': año_torneo,
                        'Tipo de Torneo': tipo_torneo
                    }

                    if fecha_nombre != 'Resultados':
                        data.append(partido)

                except Exception as e:
                    print(f"Error procesando partido en {fecha_nombre}: {e}")
                    continue

    if not data:
        print("No se encontraron datos para procesar")
        return None

    df = pd.DataFrame(data)

    # Verificaciones
    num_fechas = len(df['Fecha'].unique())
    partidos_por_fecha = df.groupby('Fecha').size()

    print(f"\nNúmero total de fechas encontradas: {num_fechas}")
    print("\nPartidos por fecha:")
    print(partidos_por_fecha)

    # Obtener el nombre de la URL para el archivo CSV
    parsed_url = urllib.parse.urlparse(url)
    path = parsed_url.path.split('/')[-1]  # Obtener la parte después de 'wiki/'
    filename = urllib.parse.unquote(path)  # Decodificar URL para obtener el nombre correcto

    # Crear la carpeta 'todos_contra_todos' si no existe
    carpeta = 'todos_contra_todos2015-2019'
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Guardar en CSV dentro de la carpeta especificada
    file_path = os.path.join(carpeta, f'{filename}.csv')
    df.to_csv(file_path, index=False)
    print(f"\nLos resultados se han guardado en '{file_path}'")

    return df

In [135]:
def procesar_urls_desde_archivo():
    # Leer las URLs desde el archivo urls.txt
    with open('url5.txt', 'r') as f:
        urls = f.readlines()

    for url in urls:
        url = url.strip()  # Eliminar posibles saltos de línea y espacios en blanco
        print(f"Procesando URL: {url}")

        df_resultados = obtener_resultados_wikipedia2015(url)

        if df_resultados is not None:
            print("\nPrimeros registros del DataFrame:")
            print(df_resultados.head())
        else:
            print(f"No se pudo obtener el DataFrame para {url}")

# Ejecutar el procesamiento de todas las URLs
procesar_urls_desde_archivo()

Procesando URL: https://es.wikipedia.org/wiki/Torneo_Finalizaci%C3%B3n_2015_(Colombia)
Tablas encontradas: 27
Procesando Fecha 1
Procesando Fecha 2
Procesando Fecha 3
Procesando Fecha 4
Procesando Fecha 5
Procesando Fecha 6
Procesando Fecha 7
Procesando Fecha 8
Procesando Fecha 9
Procesando Fecha 10
Procesando Fecha 11
Procesando Fecha 12
Procesando Fecha 13
Procesando Fecha 14
Procesando Fecha 15
Procesando Fecha 16
Procesando Fecha 17
Procesando Fecha 18
Procesando Fecha 19
Procesando Fecha 20

Número total de fechas encontradas: 20

Partidos por fecha:
Fecha
Fecha 1     10
Fecha 10    10
Fecha 11    10
Fecha 12    10
Fecha 13    10
Fecha 14    10
Fecha 15    10
Fecha 16    10
Fecha 17    10
Fecha 18    10
Fecha 19    10
Fecha 2     10
Fecha 20    10
Fecha 3     10
Fecha 4     10
Fecha 5     10
Fecha 6     10
Fecha 7     10
Fecha 8     10
Fecha 9     10
dtype: int64

Los resultados se han guardado en 'todos_contra_todos2015-2019/Torneo_Finalización_2015_(Colombia).csv'

Primeros regi

In [140]:
url = "https://es.wikipedia.org/wiki/Torneo_Apertura_2019_(Colombia)"

print("Iniciando script...")
# Obtener la información
df_resultados = obtener_resultados_wikipedia2015(url)

if df_resultados is not None:
    print("\nPrimeros registros del DataFrame:")
    print(df_resultados.head())
else:
    print("No se pudo obtener el DataFrame")

Iniciando script...
Tablas encontradas: 32
Procesando Fecha 1
Procesando Fecha 2
Procesando Fecha 3
Procesando Fecha 4
Procesando Fecha 5
Procesando Fecha 6
Procesando Fecha 7
Procesando Fecha 8
Procesando Fecha 9
Procesando Fecha 10
Procesando Fecha 11
Procesando Fecha 12
Procesando Fecha 13
Procesando Fecha 14
Procesando Fecha 15
Procesando Fecha 16
Procesando Fecha 17
Procesando Fecha 18
Procesando Fecha 19
Procesando Fecha 20
Procesando Resultados
Procesando Resultados

Número total de fechas encontradas: 20

Partidos por fecha:
Fecha
Fecha 1     10
Fecha 10    10
Fecha 11    10
Fecha 12    10
Fecha 13    10
Fecha 14    10
Fecha 15    10
Fecha 16    10
Fecha 17    10
Fecha 18    10
Fecha 19    10
Fecha 2     10
Fecha 20    10
Fecha 3     10
Fecha 4     10
Fecha 5     10
Fecha 6     10
Fecha 7     10
Fecha 8     10
Fecha 9     10
dtype: int64

Los resultados se han guardado en 'todos_contra_todos2015-2019/Torneo_Apertura_2019_(Colombia).csv'

Primeros registros del DataFrame:
     F

In [141]:
# prompt: quiero descargar todo lo que tengo en la carpeta todos_contra_todos

import shutil
import os

# Define la carpeta que quieres descargar
carpeta_a_descargar = "todos_contra_todos2015-2019"

# Verifica si la carpeta existe
if os.path.exists(carpeta_a_descargar):
  # Crea un archivo zip de la carpeta
  shutil.make_archive(carpeta_a_descargar, 'zip', carpeta_a_descargar)

  # Descarga el archivo zip
  from google.colab import files
  files.download(f"{carpeta_a_descargar}.zip")

  print(f"La carpeta '{carpeta_a_descargar}' se ha descargado como '{carpeta_a_descargar}.zip'")
else:
  print(f"La carpeta '{carpeta_a_descargar}' no existe.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

La carpeta 'todos_contra_todos2015-2019' se ha descargado como 'todos_contra_todos2015-2019.zip'


ME FALTA:
* 2002-2003 (NO HAY DATA)
* 2010 I-II (POR LAS TABLAS ESTA DURO DE SACAR, MEJOR MANUAL)
* TODOS LOS TORNEOS LARGOS


### Merge:

In [40]:
# prompt: junta todos los csv de todos contra todos, de 2004 a 2009 y de 2011 a 2024 (finalmente seran 2 df)

import pandas as pd
import glob
import os

def combine_csv_files(folder_path, years):
    """Combines all CSV files in a folder for specified years into a single DataFrame."""

    all_files = glob.glob(os.path.join(folder_path, "*.csv"))

    dfs = []
    for filename in all_files:
        try:
          # Extract the year from the filename (assuming a consistent naming convention)
          year_str = filename.split("_")[-2].split("(")[0]
          if not year_str.isdigit():
            year_str = filename.split("_")[-2] #Alternative for some files
            if not year_str.isdigit():
              year_str = filename.split("_")[-2].split(")")[0] #Alternative for some files

          if not year_str.isdigit():
            continue

          year = int(year_str)
          if year in years:
              df = pd.read_csv(filename)
              dfs.append(df)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

    if not dfs:
        print("No files found for the given years.")
        return None

    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

# Specify the folder containing your CSV files
folder_path = "todos_contra_todos"  # Replace with your actual folder path

# Define the year ranges
years_2004_2009 = range(2004, 2010)
years_2011_2024 = range(2011, 2025)

# Combine the CSV files for each range
df_2004_2009 = combine_csv_files(folder_path, years_2004_2009)
df_2011_2024 = combine_csv_files(folder_path, years_2011_2024)

# Check if DataFrames were created successfully
if df_2004_2009 is not None:
    print("DataFrame for 2004-2009 created successfully.")
    # Now you can work with df_2004_2009
    print(df_2004_2009.head())
    #print(df_2004_2009.info())
else:
  print("Error creating DataFrame for 2004-2009")

if df_2011_2024 is not None:
    print("DataFrame for 2011-2024 created successfully.")
    # Now you can work with df_2011_2024
    print(df_2011_2024.head())
    #print(df_2011_2024.info())
else:
  print("Error creating DataFrame for 2011-2024")

No files found for the given years.
Error creating DataFrame for 2004-2009
DataFrame for 2011-2024 created successfully.
     Fecha                   Local Resultado             Visitante  \
0  Fecha 1          Envigado F. C.     1 : 3  Atlético Bucaramanga   
1  Fecha 1          Atlético Huila     2 : 1       Unión Magdalena   
2  Fecha 1  Independiente Medellín     2 : 2            La Equidad   
3  Fecha 1                  Junior     0 : 1       Águilas Doradas   
4  Fecha 1         Deportivo Pasto     0 : 0           Millonarios   

                          Estadio   Año Tipo de Torneo  
0               Polideportivo Sur  2023   Finalización  
1          Guillermo Plazas Alcid  2023   Finalización  
2               Atanasio Girardot  2023   Finalización  
3  Metropolitano Roberto Meléndez  2023   Finalización  
4          Departamental Libertad  2023   Finalización  


In [49]:
df_2011_2024_sorted = df_2011_2024.sort_values(by=['Año', 'Fecha'], ascending=[True, True])

In [50]:
df_2011_2024_sorted

,Fecha,Local,Resultado,Visitante,Estadio,Año,Tipo de Torneo
160,Fecha 1,Independiente Medellín,0 : 3,Itagüí,Atanasio Girardot,2011,Finalización
161,Fecha 1,Cúcuta Deportivo,0 : 1,Deportivo Cali,General Santander,2011,Finalización
162,Fecha 1,Santa Fe,3 : 0,Deportes Tolima,Nemesio Camacho El Campín,2011,Finalización
163,Fecha 1,Envigado F. C.,1 : 3,Atlético Nacional,Atanasio Girardot,2011,Finalización
164,Fecha 1,Once Caldas,3 : 1,Boyacá Chicó,Palogrande,2011,Finalización
...,...,...,...,...,...,...,...
1692,Fecha 9,Alianza F. C.,1 : 1,Fortaleza CEIF,Armando Maestre Pavajeau,2024,Finalización
1693,Fecha 9,Deportivo Pasto,1 : 0,Independiente Medellín,Departamental Libertad,2024,Finalización
1694,Fecha 9,Santa Fe,3 : 1,Boyacá Chicó,Nemesio Camacho El Campín,2024,Finalización
1695,Fecha 9,Junior,3 : 1,Deportivo Pereira,Metropolitano Roberto Meléndez,2024,Finalización


In [51]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_2011_2024_sorted)

https://docs.google.com/spreadsheets/d/1mPyEeRxNYw01ZZd6or76hi8k1VDVUIb504St8FFGJyk#gid=0


In [52]:
import pandas as pd
import glob
import os

def combine_csv_files(folder_path, years):
    """Combines all CSV files in a folder for specified years into a single DataFrame."""

    all_files = glob.glob(os.path.join(folder_path, "*.csv"))
    print(f"\nBuscando archivos para años {min(years)}-{max(years)}")
    print(f"Total archivos encontrados: {len(all_files)}")

    dfs = []
    for filename in all_files:
        base_filename = os.path.basename(filename)
        print(f"\nProcesando archivo: {base_filename}")

        try:
            # Intentar diferentes patrones para extraer el año
            year = None

            # Dividir el nombre del archivo por partes
            parts = base_filename.replace('(', '_').replace(')', '_').split('_')

            # Buscar una parte que sea un número de 4 dígitos
            for part in parts:
                if part.isdigit() and len(part) == 4:
                    year = int(part)
                    break

            if year is None:
                print(f"No se pudo extraer el año del archivo: {base_filename}")
                continue

            print(f"Año extraído: {year}")

            if year in years:
                print(f"Archivo válido para el rango de años solicitado")
                df = pd.read_csv(filename)
                # Agregar columna de año si no existe
                if 'year' not in df.columns:
                    df['year'] = year
                dfs.append(df)
            else:
                print(f"Archivo fuera del rango de años solicitado")

        except Exception as e:
            print(f"Error procesando {filename}: {str(e)}")

    if not dfs:
        print(f"\nNo se encontraron archivos para los años {min(years)}-{max(years)}")
        return None

    print(f"\nCombinando {len(dfs)} archivos...")
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

# Especificar la carpeta que contiene los archivos CSV
folder_path = "todos_contra_todos"

# Definir los rangos de años
years_2004_2009 = set(range(2004, 2010))
years_2011_2024 = set(range(2011, 2025))

# Combinar los archivos CSV para cada rango
print("\nProcesando archivos 2004-2009...")
df_2004_2009 = combine_csv_files(folder_path, years_2004_2009)

print("\nProcesando archivos 2011-2024...")
df_2011_2024 = combine_csv_files(folder_path, years_2011_2024)

# Verificar si los DataFrames se crearon exitosamente
if df_2004_2009 is not None:
    print("\nDataFrame 2004-2009 creado exitosamente")
    print(f"Dimensiones: {df_2004_2009.shape}")
    print("\nPrimeras filas:")
    print(df_2004_2009.head())
else:
    print("\nError creando DataFrame 2004-2009")

# if df_2011_2024 is not None:
#     print("\nDataFrame 2011-2024 creado exitosamente")
#     print(f"Dimensiones: {df_2011_2024.shape}")
#     print("\nPrimeras filas:")
#     print(df_2011_2024.head())
# else:
#     print("\nError creando DataFrame 2011-2024")


Procesando archivos 2004-2009...

Buscando archivos para años 2004-2009
Total archivos encontrados: 23

Procesando archivo: Torneo_Apertura_2008_(Colombia)_legacy.csv
Año extraído: 2008
Archivo válido para el rango de años solicitado

Procesando archivo: Torneo_Finalización_2023_(Colombia).csv
Año extraído: 2023
Archivo fuera del rango de años solicitado

Procesando archivo: Torneo_Finalización_2011_(Colombia).csv
Año extraído: 2011
Archivo fuera del rango de años solicitado

Procesando archivo: Torneo_Apertura_2013_(Colombia).csv
Año extraído: 2013
Archivo fuera del rango de años solicitado

Procesando archivo: Torneo_Finalización_2021_(Colombia).csv
Año extraído: 2021
Archivo fuera del rango de años solicitado

Procesando archivo: Torneo_Finalización_2005_(Colombia)_legacy.csv
Año extraído: 2005
Archivo válido para el rango de años solicitado

Procesando archivo: Torneo_Finalización_2008_(Colombia)_legacy.csv
Año extraído: 2008
Archivo válido para el rango de años solicitado

Proces

In [53]:
df_2004_2009

,Fecha,Día,Local,Resultado,Visitante,Año,Tipo de Torneo,year
0,Fecha 1,2 de febrero,Millonarios,1 - 2,Medellín,2008,Apertura,2008
1,Fecha 1,2 de febrero,Atlético Nacional,0 - 1,Santa Fe,2008,Apertura,2008
2,Fecha 1,3 de febrero,América de Cali,1 - 1,Huila,2008,Apertura,2008
3,Fecha 1,3 de febrero,Pereira,0 - 2,Envigado,2008,Apertura,2008
4,Fecha 1,3 de febrero,Cúcuta,0 - 1,Bucaramanga,2008,Apertura,2008
...,...,...,...,...,...,...,...,...
987,Fecha 18,7 de noviembre,Pereira,1 - 1,Medellín,2004,Finalización,2004
988,Fecha 18,7 de noviembre,Pasto,3 - 2,Huila,2004,Finalización,2004
989,Fecha 18,7 de noviembre,Nacional,2 - 0,Once Caldas,2004,Finalización,2004
990,Fecha 18,7 de noviembre,Bogotá Chicó,2 - 2,Santa Fe,2004,Finalización,2004


In [54]:
df_2004_2009_sorted = df_2004_2009.sort_values(by=['Año', 'Fecha'], ascending=[True, True])

In [55]:
df_2004_2009_sorted

,Fecha,Día,Local,Resultado,Visitante,Año,Tipo de Torneo,year
668,Fecha 1,7 de febrero,Millonarios,2 - 0,Cortuluá,2004,Apertura,2004
669,Fecha 1,8 de febrero,Bogotá Chicó,1 - 1,Junior,2004,Apertura,2004
670,Fecha 1,8 de febrero,Pasto,1 - 0,Santa Fe,2004,Apertura,2004
671,Fecha 1,8 de febrero,América,4 - 0,Deportes Quindío,2004,Apertura,2004
672,Fecha 1,8 de febrero,Envigado F. C.,1 - 1,Cali,2004,Apertura,2004
...,...,...,...,...,...,...,...,...
396,Fecha 9,21 de septiembre,Boyacá Chicó,1 - 0,Quindío,2008,Finalización,2008
397,Fecha 9,21 de septiembre,Cúcuta,0 - 0,Bucaramanga,2008,Finalización,2008
398,Fecha 9,21 de septiembre,Junior,1 - 0,Envigado,2008,Finalización,2008
399,Fecha 9,21 de septiembre,Pasto,0 - 0,La Equidad,2008,Finalización,2008


In [107]:
!echo "# Scraping_FPC_Wikipedia" >> README.md

In [108]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [109]:
!git add .

In [112]:
!git commit -m "first commit"

[master (root-commit) 58ce9e8] first commit
 91 files changed, 62153 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2025.01.15/14.24.33.570214.log
 create mode 100644 .config/logs/2025.01.15/14.24.53.360583.log
 create mode 100644 .config/logs/2025.01.15/14.25.00.998106.log
 create mode 100644 .config/logs/2025.01.15/14.25.02.096828.log
 create mode 100644 .config/logs/2025.01.15/14.25.10.736082.log
 create mode 100644 .config/logs/2025.01.15/14.25.11.351475.log
 create mode 100644 README.md
 create mode 100644 prueba/Camp

In [113]:
!git branch -M main

In [114]:
!git remote add origin git@github.com:dsrojo10/Scraping_FPC_Wikipedia.git

In [119]:
!ssh-keyscan github.com >> /root/.ssh/known_hosts


# github.com:22 SSH-2.0-427ca60ef
# github.com:22 SSH-2.0-427ca60ef
# github.com:22 SSH-2.0-427ca60ef
# github.com:22 SSH-2.0-427ca60ef
# github.com:22 SSH-2.0-427ca60ef


In [120]:
!git push -u origin main

Enumerating objects: 98, done.
Counting objects: 100% (98/98), done.
Delta compression using up to 2 threads
Compressing objects: 100% (90/90), done.
Writing objects: 100% (98/98), 8.57 MiB | 1.64 MiB/s, done.
Total 98 (delta 43), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (43/43), done.
To github.com:dsrojo10/Scraping_FPC_Wikipedia.git
 * [new branch]      main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
